In [207]:
import psycopg2

In [208]:
try:
    conn = psycopg2.connect("dbname='postgres' user='devuser' host='localhost' password='devuser'")
except:
    print("I am unable to connect to the database")

In [209]:
cur = conn.cursor()

In [220]:
cur.execute("ROLLBACK")
conn.commit()

In [211]:
cur.execute("""
DROP SCHEMA public CASCADE;
CREATE SCHEMA public;
""")
conn.commit()

In [212]:
cur.execute("""
CREATE TABLE folders (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    parent INT NULL REFERENCES folders(id))
""")

conn.commit()

In [213]:
cur.execute("""
CREATE TABLE modules (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    parent INT REFERENCES folders(id))
""")

conn.commit()

In [214]:
cur.execute("""
INSERT INTO folders (id, name)
VALUES (DEFAULT, '{folder_name}')
RETURNING id
""".format(folder_name="Folder1"))

folder_1_id = cur.fetchone()[0]

conn.commit()

cur.execute("""
INSERT INTO folders (id, name)
VALUES (DEFAULT, '{folder_name}')
RETURNING id
""".format(folder_name="Folder2"))

folder_2_id = cur.fetchone()[0]

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
RETURNING id
""".format(folder_name="Folder11", parent_folder_id=folder_1_id))

folder_1_1_id = cur.fetchone()[0]

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
""".format(folder_name="Folder111", parent_folder_id=folder_1_1_id))

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
RETURNING id
""".format(folder_name="Folder21", parent_folder_id=folder_2_id))

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
RETURNING id
""".format(folder_name="Folder22", parent_folder_id=folder_2_id))

conn.commit()

In [215]:
new_module_name = "Module3"
new_module_parent = folder_2_id

cur.execute("""
INSERT INTO modules (id, name, parent)
VALUES (DEFAULT, '{module_name}', {parent_folder_id})
RETURNING id
""".format(module_name=new_module_name, parent_folder_id=new_module_parent))

new_module_id = cur.fetchone()[0]

new_module_table_name = "module_" + str(new_module_id) + "_" + new_module_name

cur.execute("""
CREATE TABLE {module_table_name} (
    id SERIAL PRIMARY KEY,
    default_column TEXT)
""".format(module_table_name=new_module_table_name))

conn.commit()

In [216]:
def print_with_indent(text, indent=0):
    for current_indent in range(0, indent):
        print("    ", end=" ")
    
    print(text)

In [217]:
def list(parent=None, indent=0):
    folders_rows = []
    modules_rows = []

    try:
        if(parent is None):
            cur.execute("""
            SELECT * FROM folders WHERE parent IS NULL
            """)

            folders_rows = cur.fetchall()
        else:
            cur.execute("""
            SELECT * FROM folders WHERE parent={query_parent}
            """.format(query_parent=parent))

            folders_rows = cur.fetchall()

            cur.execute("""
            SELECT * FROM modules WHERE parent={query_parent}
            """.format(query_parent=parent))

            modules_rows = cur.fetchall()
    except:
        print("I cannot execute")

    for folder in folders_rows:
        folder_id = folder[0]
        folder_name = folder[1]
        folder_parent = folder[2]

        print_with_indent(folder_name + "\n", indent)

        list(folder_id, indent + 1)

    for module in modules_rows:
        module_id = module[0]
        module_name = module[1]
        module_parent = module[2]

        print_with_indent(module_name + "\n", indent)    

list()

Folder1

     Folder11

          Folder111

Folder2

     Folder21

     Folder22

     Module3



In [221]:
new_column_name="New Column"

cur.execute("""
ALTER TABLE {module_table_name}
ADD "{column_name}" TEXT;
""".format(
    module_table_name=new_module_table_name,
    column_name=new_column_name))

conn.commit()